In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import io
from image_function import *
from model import *

Using TensorFlow backend.


In [2]:
main_folder = 'Data\\sample3\\'

In [3]:
Path_full_size = main_folder + 'pad\\'
Path_crop = main_folder + 'temp\\'
Path_predict= main_folder + 'Predict\\'
N_pieces = 1024 # crop to this number of pieces before upsampling

# each image is 1024 x 1024,  it is cropped to 1024 32 x 32 pieces
# then upsampled to 256 x 256 for prediciton

In [4]:
ensure_dir(Path_crop+'\\') # create folder if not exist
ensure_dir(Path_predict) # create folder if not exist

# Predict results using the trained U-net model

#### Step 1: Crop the original images to small pieces, in total there are 52 samples

#### Step 2: Read the crop images for one sample,  then upsample them to the shape for the network, in this example, the target size is 256 x 256

#### Step 3: Create prediction results of one sample 

#### Step 4: Stitch it back to a whole images

In [5]:
def testGenerator(test_path,N_slices, N_image_start, N_image_stop,target_size = (256,256),flag_multi_class = False,as_gray = True):
    # input:
    # test_path: path of the images to read
    # N_slices -  number of slice per images
    # N_image_start - index of the first image to read
    # N_image_stop - index of the last image to read 
    # target_size - shape of the images for output
    file_list = create_filename(N_slices, N_image_start, N_image_stop)
    for filename in file_list:
        f = os.path.join(test_path,filename)
        img =io.imread(f, as_gray = as_gray)

        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)

       # img = np.expand_dim(img, axis = 2)
        yield img

In [6]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.jpg"%i),img)

In [7]:
images_n = read_crop_write(Path_full_size, Path_crop, N_pieces) 
# crop full size images to small patches

In [8]:
model = unet()
model.load_weights("unet.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\yizhe\Desktop\Yi\FP03.006Demo\model.py:67: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = outputs)


In [9]:
for i in range(images_n):
    N_image_start = i
    N_image_stop = N_image_start
    testGene = testGenerator(Path_crop,N_pieces, N_image_start, N_image_stop)
    results = model.predict_generator(testGene, N_pieces, verbose = 1)
    images = np.squeeze(results)
    whole_img = stitch_image (images, N_pieces)
    whole_img = trans.resize(whole_img,[1024, 1024])
    
    filename = Path_predict + str(N_image_start)+'.jpg'
    cv2.imwrite(filename,whole_img*100)

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


1024/1024 [==============================] - 6s 6ms/step
